In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dataworkshop/dw_road_signs'

/content/drive/My Drive/Colab Notebooks/dataworkshop/dw_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)

if y_test.ndim == 1:
  y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(512, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')                    
  ])

In [0]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit={})

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.8574 - accuracy: 0.2283 - val_loss: 1.3792 - val_accuracy: 0.5442
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 1.2497 - accuracy: 0.5942 - val_loss: 0.4498 - val_accuracy: 0.8634
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.6452 - accuracy: 0.7917 - val_loss: 0.1713 - val_accuracy: 0.9534
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3907 - accuracy: 0.8768 - val_loss: 0.0930 - val_accuracy: 0.9720
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2659 - accuracy: 0.9181 - val_loss: 0.0560 - val_accuracy: 0.9828


0.9673469387755103

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(params.get('dropout_cnn_block_0', 0.3)),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(params.get('dropout_cnn_block_1', 0.3)),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(params.get('dropout_cnn_block_2', 0.3)),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params.get('dropout_dense_block_0', 0.3)),

    Dense(512, activation='relu'),
    Dropout(params.get('dropout_dense_block_1', 0.3)),

    Dense(num_classes, activation='softmax')                    
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=params.get('epochs', 5),
      verbose=0
      )
  
  accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
  print(f'{params}\naccuracy={accuracy}\n')

  return {'loss': -accuracy, 
          'status': STATUS_OK, 
          'model': model}


In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),

    'dropout_cnn_block_0': hp.uniform('dropout_cnn_block_0', 0.3, 0.5),
    'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
    'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),

    'dropout_dense_block_0': hp.uniform('dropout_dense_block_0', 0.3, 0.7),
    'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.7),
}

trials = Trials()
best = fmin(
    fn=func_obj,
    space=space,
    algo=tpe.suggest,
    max_evals=30,
    trials=trials
)

{'batch_size': 120.0, 'dropout_cnn_block_0': 0.46427927322626344, 'dropout_cnn_block_1': 0.4435743822907009, 'dropout_cnn_block_2': 0.48887347122754077, 'dropout_dense_block_0': 0.6824972217113041, 'dropout_dense_block_1': 0.5812314647065218}
accuracy=0.7891156673431396

{'batch_size': 160.0, 'dropout_cnn_block_0': 0.41740058499130034, 'dropout_cnn_block_1': 0.37281531812717217, 'dropout_cnn_block_2': 0.34334729901967476, 'dropout_dense_block_0': 0.5979827412234566, 'dropout_dense_block_1': 0.5166111668896176}
accuracy=0.866439938545227

{'batch_size': 170.0, 'dropout_cnn_block_0': 0.48213569623300606, 'dropout_cnn_block_1': 0.3268580988699837, 'dropout_cnn_block_2': 0.38088659730770746, 'dropout_dense_block_0': 0.5133126809906665, 'dropout_dense_block_1': 0.4032028531835275}
accuracy=0.9383220076560974

{'batch_size': 140.0, 'dropout_cnn_block_0': 0.37049310573892136, 'dropout_cnn_block_1': 0.3527402169572835, 'dropout_cnn_block_2': 0.3856841799628709, 'dropout_dense_block_0': 0.69204

In [0]:
idx = np.argmin(trials.losses())
best_model = trials.results[idx]['model']
best_model.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1234 - accuracy: 0.9676


[0.12342412769794464, 0.9675737023353577]